## Deliverable 2. Create a Customer Travel Destinations Map.

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [42]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Soweto,ZA,-26.2678,27.8585,52.77,21,0,6.89,clear sky
1,1,San Carlos De Bariloche,AR,-41.1456,-71.3082,42.55,56,75,9.13,broken clouds
2,2,Bluff,NZ,-46.6000,168.3333,48.25,69,90,7.58,overcast clouds
3,3,Bredasdorp,ZA,-34.5322,20.0403,54.81,69,0,2.19,clear sky
4,4,Tevaitoa,PF,-16.7833,-151.5000,81.14,71,85,8.90,overcast clouds


In [43]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [44]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Tevaitoa,PF,-16.7833,-151.5000,81.14,71,85,8.90,overcast clouds
12,12,Hilo,US,19.7297,-155.0900,80.28,75,1,5.75,clear sky
15,15,Kapaa,US,22.0752,-159.3190,77.32,78,40,1.01,scattered clouds
18,18,Albany,US,42.6001,-73.9662,87.69,52,21,3.00,few clouds
22,22,Puerto Ayora,EC,-0.7393,-90.3518,82.36,84,96,7.00,overcast clouds


In [45]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [46]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Tevaitoa,PF,-16.7833,-151.5000,81.14,71,85,8.90,overcast clouds
12,12,Hilo,US,19.7297,-155.0900,80.28,75,1,5.75,clear sky
15,15,Kapaa,US,22.0752,-159.3190,77.32,78,40,1.01,scattered clouds
18,18,Albany,US,42.6001,-73.9662,87.69,52,21,3.00,few clouds
22,22,Puerto Ayora,EC,-0.7393,-90.3518,82.36,84,96,7.00,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
662,662,Cheyenne,US,41.1400,-104.8203,86.97,25,1,11.50,clear sky
663,663,Mtwara,TZ,-10.2667,40.1833,75.24,76,13,13.20,few clouds
665,665,Fonte Boa,BR,-2.4959,-66.2662,82.56,86,32,1.86,scattered clouds
671,671,Karaton,ID,-6.3424,106.1045,81.95,94,53,2.73,broken clouds


In [47]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tevaitoa,PF,81.14,overcast clouds,-16.7833,-151.5000,
12,Hilo,US,80.28,clear sky,19.7297,-155.0900,
15,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,
18,Albany,US,87.69,few clouds,42.6001,-73.9662,
22,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,
24,Snyder,US,78.53,broken clouds,32.7179,-100.9176,
33,Rikitea,PF,75.58,overcast clouds,-23.1203,-134.9692,
36,Jamestown,US,84.09,clear sky,42.0970,-79.2353,
47,Butaritari,KI,82.40,scattered clouds,3.0707,172.7902,
50,Fare,PF,80.49,overcast clouds,-16.7000,-151.0167,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels= requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df[hotel_df['Hotel Name']!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Tevaitoa,PF,81.14,overcast clouds,-16.7833,-151.5000,Hôtel Raiatea Lodge
12,Hilo,US,80.28,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
15,Kapaa,US,77.32,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
22,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
24,Snyder,US,78.53,broken clouds,32.7179,-100.9176,"Holiday Inn Express & Suites Snyder, an IHG Hotel"
33,Rikitea,PF,75.58,overcast clouds,-23.1203,-134.9692,Pension Maro'i
36,Jamestown,US,84.09,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
47,Butaritari,KI,82.40,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
50,Fare,PF,80.49,overcast clouds,-16.7000,-151.0167,Maitai Lapita Village Huahine
52,Kahului,US,77.85,few clouds,20.8947,-156.4700,Maui Seaside Hotel


In [56]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [62]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [66]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0),zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))